In [23]:
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import FreqDist
import tf_glove
import pickle
%matplotlib  inline

Learn embedings with Glove algorithm

In [2]:
train_df = pd.read_pickle('Tra.p')

In [21]:
lemmatizer = WordNetLemmatizer()
def convert_to_lem(text):
    tokens = []
    for word in  nltk.word_tokenize(text, language='english'):
        #if len(word) < 2:
            #continue
        tokens.append(lemmatizer.lemmatize(word.lower()))
    return tokens

def tokenize_on_whitespace(sent):
    result = []
    for w in WhitespaceTokenizer().tokenize(sent):
        result.append(w.lower())
    return result
def word_sort_by_freq(series, threshold):
    threshold += 1
    df_array = series.values.tolist()
    flattend = []
    for row in df_array:
        flattend += row
    dist = FreqDist(flattend)
    sortedToken = sorted(list(set(flattend)), key=lambda token: dist[token], reverse=False)
    word_freq_sorted = [(token, dist[token]) for token in sortedToken]

    for idx in range(len(word_freq_sorted)):
        if word_freq_sorted[idx][1] == threshold:
            start = idx
            break
    return  word_freq_sorted[start:]
    
def word_with_freq_above_treshold(series, threshold=4):
    word_freq_sorted = word_sort_by_freq(series, threshold)
    result = [line[0] for line in word_freq_sorted]    
    return result

In [10]:
tokens = train_df['Text'].apply(convert_to_lem)
tokens = tokens.apply(lambda x: len(x))
tokens.plot.hist()
print(tokens.sort_values(ascending=False).head(10))
max_num_words = (tokens.max())

150164    52
387521    52
79671     46
151256    45
479672    44
214038    43
184062    42
8766      41
283195    40
5906      40
Name: Text, dtype: int64


In [12]:
pad = ["abcde", "abcde", "abcde", "abcde", "abcde"]
def add_pad(tkn):
    return tkn + pad
def Glove_TF(text, lemas=True):
    """Train word 2 vec"""
    if lemas:
        corp = text.apply(convert_to_lem)
    else:
        corp = text.apply(tokenize_on_whitespace)
    corp = corp.apply(add_pad)
    
    model  =  tf_glove.GloVeModel(embedding_size=50, context_size=5, min_occurrences=5,
                                learning_rate=0.05, batch_size=256)    
    model.fit_to_corpus(corp)
    model.train(num_epochs=1, summary_batch_interval=512)
    return model

In [13]:
norm_m = Glove_TF(train_df['Text'], False)

In [14]:
lema_m = Glove_TF(train_df['Text'])

In [25]:
def conv_to_idx(text, model, lemas=True):
    if lemas:
        corp = text.apply(convert_to_lem)
    else:
        corp = text.apply(tokenize_on_whitespace)
    corp = corp.apply(add_pad)

    word_above_tresh = word_with_freq_above_treshold(corp, 4)
    i = 0
    
    my_words_to_index = {}
    my_index_to_words = {}
    my_word_to_vec_map = {}
    for w in sorted(word_above_tresh):
        i = i + 1
        my_words_to_index[w] = i
        my_index_to_words[i] = w
        my_word_to_vec_map[w] = model.embedding_for(w)
    return my_words_to_index, my_index_to_words, my_word_to_vec_map

In [26]:
my_words_to_index_low, my_index_to_words_low, my_word_to_vec_map_low = conv_to_idx(train_df['Text'],norm_m, False)

In [ ]:
#pickle.dump(my_word_to_vec_map_low, open( "w2v_low_map.p", "wb" ) )

In [28]:
my_words_to_index_lem, my_index_to_words_lem, my_word_to_vec_map_lem = conv_to_idx(train_df['Text'],lema_m)

In [ ]:
#pickle.dump(my_words_to_index_lem, open( "w2v_low_map.p", "wb" ) )